In [1]:
import pandas as pd
import mysql.connector

In [2]:
nuevas_ofertas = pd.read_csv("../csv/PARTE_1_2_df_actualizacion_limpio.csv",
                             dtype={"salario_min": "float64",
                                    "salario_max": "float64",
                                    "estudios": "bool",
                                    "experiencia": "float64",
                                    "vacaciones": "Int64"},
                             parse_dates=["fecha"])

In [3]:
nuevas_ofertas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_urls                996 non-null    object        
 1   fecha                  996 non-null    datetime64[ns]
 2   oferta                 996 non-null    object        
 3   empresa                996 non-null    object        
 4   funcion                974 non-null    object        
 5   salario_min            221 non-null    float64       
 6   salario_max            221 non-null    float64       
 7   estudios               996 non-null    bool          
 8   experiencia            975 non-null    float64       
 9   ubicacion              980 non-null    object        
 10  modalidad              532 non-null    object        
 11  jornada                995 non-null    object        
 12  contrato               983 non-null    object        
 13  skill

In [4]:
nuevas_ofertas.columns

Index(['id_urls', 'fecha', 'oferta', 'empresa', 'funcion', 'salario_min',
       'salario_max', 'estudios', 'experiencia', 'ubicacion', 'modalidad',
       'jornada', 'contrato', 'skills', 'tecnologias_aptitudes', 'idiomas',
       'vacaciones', 'beneficios', 'portal_empleo', 'descripcion'],
      dtype='object')

In [5]:
db = mysql.connector.connect(host="localhost",
                             user="root",
                             password="contraseña1234",
                             database="ofertas_empleo")
cursor = db.cursor()

In [6]:
#FUNCIONES: VALORES EXISTENTES / INSETAR NUEVOS VALORES / OBTENER IDS / INSERTAR EN OFERTAS

#valores existentes
def valores_existentes(tabla, columna):
    cursor.execute(f"SELECT {columna} FROM {tabla}")
    return set(row[0] for row in cursor.fetchall())


#insertar nuevos valores
def insertar_nuevos_valores(tabla, columna, nuevos_valores):
    for val in nuevos_valores:
        cursor.execute(f"INSERT INTO {tabla} ({columna}) VALUES (%s)", (val,))
    db.commit()


#obtener el mapeo actualizado de los IDS
def obtener_id_map(tabla, id_col, val_col):
    cursor.execute(f"SELECT {id_col}, {val_col} FROM {tabla}")
    return {row[1]: row[0] for row in cursor.fetchall()}


#hacemos una funcion que limpie los valores nulos
def clean_for_mysql(df):
    # Convierte cualquier valor nulo (NaN, pd.NA) a None
    return df.applymap(lambda x: None if pd.isna(x) else x)


#insertar datos en la tabla pricipal
def insertar_tablaprincipal_ofertas(ofertas_df):
    insert_query = """INSERT INTO ofertas (
        id_urls, fecha, oferta, id_empresa, funcion, salario_min, salario_max, estudios, experiencia,
        id_ubicacion, id_modalidad, id_jornada, id_contrato, vacaciones, id_portal_empleo, descripcion
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    for _, row in ofertas_df.iterrows():
        valores = [
            row["id_urls"],
            row["fecha"],
            row["oferta"],
            row["id_empresa"],
            row["funcion"],
            row["salario_min"],
            row["salario_max"],
            row["estudios"],
            row["experiencia"],
            row["id_ubicacion"],
            row["id_modalidad"],
            row["id_jornada"],
            row["id_contrato"],
            row["vacaciones"],
            row["id_portal_empleo"],
            row["descripcion"]
        ]
        #convertir explicitamente los de nulos y tipos
        valores = [None if pd.isna(x) else x for x in valores]
        cursor.execute(insert_query, tuple(valores))
    db.commit()


#insertar datos en las tablas de relacion
def insertar_tablas_relacion(tabla_relacion, id_col_relacion, id_col_valor, relaciones):
    # relaciones es lista de tuplas (id_urls, id_valor)
    insert_query = f"INSERT IGNORE INTO {tabla_relacion} (id_urls, {id_col_valor}) VALUES (%s, %s)"
    cursor.executemany(insert_query, relaciones)
    db.commit()

In [7]:
#ACTUALIZAMOS LOS DATOS

#actulizamos en las tablas de referencia 
tablas_referencia = {"empresa": "empresa",
                     "ubicacion": "ubicacion",
                     "modalidad": "modalidad",
                     "jornada": "jornada",
                     "contrato": "contrato",
                     "portal_empleo": "portal_empleo"}

for tabla, columna in tablas_referencia.items():
    existentes = valores_existentes(tabla, columna)
    nuevos = set(nuevas_ofertas[columna].dropna().unique()) - existentes
    if nuevos:
        insertar_nuevos_valores(tabla, columna, nuevos)
        print(f"Insertadas {len(nuevos)} nuevas filas en {tabla}")


#actualizamos en las tablas con valores multiples (skills, tecnologias_aptitudes, idiomas, beneficios)
tablas_multi = {"skills": "skills",
                "tecnologias_aptitudes": "tecnologias_aptitudes",
                "idiomas": "idiomas",
                "beneficios": "beneficios"}

for tabla, columna in tablas_multi.items():
    existentes = valores_existentes(tabla, columna)
    # Extraer lista de valores separados por coma y aplanar
    lista_valores = nuevas_ofertas[columna].dropna().apply(lambda x: [v.strip() for v in x.split(',')]).sum()
    nuevos = set(lista_valores) - existentes
    if nuevos:
        insertar_nuevos_valores(tabla, columna, nuevos)
        print(f"Insertadas {len(nuevos)} nuevas filas en {tabla}")


#actualizamos los mapeos de las IDS en las tablas con valores unicos
map_empresa = obtener_id_map("empresa", "id_empresa", "empresa")
map_ubicacion = obtener_id_map("ubicacion", "id_ubicacion", "ubicacion")
map_modalidad = obtener_id_map("modalidad", "id_modalidad", "modalidad")
map_jornada = obtener_id_map("jornada", "id_jornada", "jornada")
map_contrato = obtener_id_map("contrato", "id_contrato", "contrato")
map_portal = obtener_id_map("portal_empleo", "id_portal_empleo", "portal_empleo")

#actualizamos los mapeos de las IDS en las tablas con valores multiples
map_skills = obtener_id_map("skills", "id_skills", "skills")
map_tecnologias = obtener_id_map("tecnologias_aptitudes", "id_tecnologias", "tecnologias_aptitudes")
map_idiomas = obtener_id_map("idiomas", "id_idiomas", "idiomas")
map_beneficios = obtener_id_map("beneficios", "id_beneficios", "beneficios")


#hacemos una copia para mapear los IDS en la tabla principal ofertas
df = nuevas_ofertas.copy()
df["id_empresa"] = df["empresa"].map(map_empresa)
df["id_ubicacion"] = df["ubicacion"].map(map_ubicacion)
df["id_modalidad"] = df["modalidad"].map(map_modalidad)
df["id_jornada"] = df["jornada"].map(map_jornada)
df["id_contrato"] = df["contrato"].map(map_contrato)
df["id_portal_empleo"] = df["portal_empleo"].map(map_portal)

Insertadas 40 nuevas filas en empresa
Insertadas 4 nuevas filas en ubicacion
Insertadas 8 nuevas filas en tecnologias_aptitudes


In [8]:
#aplicacmos la funcion de limpieza de nulos
df = clean_for_mysql(df)

#booleanos a int (0/1)
df["estudios"] = df["estudios"].astype(int)

#fechas a string en formato
df["fecha"] = pd.to_datetime(df["fecha"]).dt.strftime('%Y-%m-%d %H:%M:%S')

#vacaciones a float (para que los nulos sean none)
df["vacaciones"] = df["vacaciones"].astype(float)

C:\Users\GoraJ\AppData\Local\Temp\ipykernel_14656\1951713169.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: None if pd.isna(x) else x)


In [9]:
#INSERCIÓN

In [10]:
#insertamos solo las ofertas nuevas
cursor.execute("SELECT id_urls FROM ofertas")
urls_existentes = set(row[0] for row in cursor.fetchall())
ofertas_nuevas = df[~df["id_urls"].isin(urls_existentes)]

if not ofertas_nuevas.empty:
    insertar_tablaprincipal_ofertas(ofertas_nuevas)
    print(f"Insertadas {len(ofertas_nuevas)} nuevas ofertas")


#hacemos una funcion para insertar las relaciones de las tablas de relaciones multiples
def preparar_relaciones(df, columna, map_columna, tabla_relacion, id_col_relacion):
    # Filtrar filas con datos y solo ofertas nuevas
    df_rel = df[["id_urls", columna]].dropna()
    df_rel = df_rel[df_rel["id_urls"].isin(ofertas_nuevas["id_urls"])]
    relaciones = []
    for _, row in df_rel.iterrows():
        valores = [v.strip() for v in row[columna].split(',')]
        for val in valores:
            id_val = map_columna.get(val)
            if id_val is not None:
                relaciones.append((row["id_urls"], id_val))
    if relaciones:
        insertar_tablas_relacion(tabla_relacion, "id_urls", id_col_relacion, relaciones)
        print(f"Insertadas {len(relaciones)} relaciones en {tabla_relacion}")

#aplicamos la funcion
preparar_relaciones(df, "skills", map_skills, "oferta_skill", "id_skills")
preparar_relaciones(df, "tecnologias_aptitudes", map_tecnologias, "oferta_tecnologia", "id_tecnologias")
preparar_relaciones(df, "idiomas", map_idiomas, "oferta_idioma", "id_idiomas")
preparar_relaciones(df, "beneficios", map_beneficios, "oferta_beneficio", "id_beneficios")


cursor.close()
db.close()

print("Actualización completada con éxito.")

Insertadas 891 nuevas ofertas
Insertadas 369 relaciones en oferta_skill
Insertadas 1730 relaciones en oferta_tecnologia
Insertadas 151 relaciones en oferta_idioma
Insertadas 243 relaciones en oferta_beneficio
Actualización completada con éxito.
